In [1]:
import pandas as pd
import networkx as nx
import pycountry as pc
import pycountry_convert as pcc
from collections import defaultdict

In [2]:
total = pd.read_csv("major-trading-partners.csv")

In [3]:
total = total[total['Year']==2018]

In [4]:
total.shape

(1272, 9)

In [5]:
total = total[
    (~total['Major trading partner 1 (% of exports)'].isin(['Areas nes',
                                                            'Bunkers', 
                                                            'Undisclosed', 
                                                            'Free Zones',
                                                            'Areas, nes',
                                                            'Asia nes',
                                                            'Europe nes']) &
     (~total['Major trading partner'].isin(['Other non-specified areas'])))
].replace('Dem. Rep. of the Congo',
          'Congo, The Democratic Republic of the').replace('Rep. of Korea',
                                                           'Korea, Republic of')

In [6]:
def get_alphacode(c):
    return pc.countries.search_fuzzy(c)[0].alpha_3

def get_continent(c):
    return pcc.country_alpha2_to_continent_code(c)

In [7]:
total['country_code'] = [get_alphacode(c) for c in total['Major trading partner']]
total['partner_code'] = [get_alphacode(c) for c in total['Major trading partner 1 (% of exports)']]

In [8]:
only_imports = ['Major trading partner 1 (% of imports)', 
                'Major trading partner 2 (% of imports)', 
                'Major trading partner 3 (% of imports)']

idx2 = ['partner_code' , 'country_code', 'Value']
imports = total[total['Series'].isin(only_imports)][idx2].copy() 

In [9]:
only_exports = ['Major trading partner 1 (% of exports)', 
                'Major trading partner 2 (% of exports)', 
                'Major trading partner 3 (% of exports)']
idx1 = ['country_code', 'partner_code' , 'Value']

exports = total[total['Series'].isin(only_exports)][idx1].copy()  

In [10]:
imports.rename(columns={
    'partner_code': 'Source',
    'country_code': 'Target',
    'Value': 'Weight'
}, inplace=True)

exports.rename(columns={
    'country_code': 'Source',
    'partner_code': 'Target',
    'Value': 'Weight'
}, inplace=True)

pd.concat([imports,exports])

,Source,Target,Weight
3,IRN,AFG,19.3620
7,PAK,AFG,18.3461
11,CHN,AFG,16.7230
17,ITA,ALB,28.6278
22,TUR,ALB,8.1093
...,...,...,...
3634,ZMB,CHE,42.0801
3640,ZMB,CHN,14.4475
3646,ZMB,COD,9.5418
3652,ZWE,ZAF,51.4752
